<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/2_objectdetection/GC2_7_SSD_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.7 学習と検証の実施

- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。


# 学習目標

1.	SSDの学習を実装できるようになる

---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

In [ ]:
!ls

In [ ]:
%cd "pytorch_advanced"

In [ ]:
!ls

In [ ]:
%cd "2_objectdetection"

In [ ]:
!ls

---
# 事前準備

- **GCの**GPUインスタンスを使用します
- フォルダ「utils」のssd_model.pyを利用します

In [ ]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

---
# VOC2012準備

VOCdevkit/ from VOCtrainval_11-May-2012.tar is placed at /root .  


In [ ]:
# It takes one minutes or so.

%cd /root
!tar xf "/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/2_objectdetection/data/VOCtrainval_11-May-2012.tar"
!ls /root/VOCdevkit
%cd -

---
# DatasetとDataLoaderを作成する

In [ ]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# ファイルパスのリストを取得
rootpath = "/root/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoaderを作成する
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデルを作成する

In [ ]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSDの初期の重みを設定
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


# 損失関数と最適化手法を定義する

In [ ]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 学習の準備

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのloss （Issue158での誤植修正）
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


# 学習の実行

[U-Tsukuba] Extra remarks  

You need probably 4-5 hours to run 50 epochs.  
Maybe 10 epochs would be sufficient to see the behavior of the training phase here.  
The spec of the runtime instance can be investigated by [100-ColabTutorial.ipynb](https://github.com/kameda-yoshinari/IMISToolExeA2021/blob/main/100_ColabTutorial.ipynb).

As of 2021/07, Intel(R) Xeon(R) CPU @ 2.20GHz x 2 + Tesla T4 (15GB) would be a rumtime instance with GPU.

(We can utilize the well-trained result (ssd300_50.pth) from the author's web site.)


In [ ]:
# 学習・検証を実行する

# Just to surpress UserWarning
import warnings
warnings.simplefilter('ignore')

num_epochs= 10  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

[U-Tsukuba] Extra remarks 

You can see the loss graph for 50 epochs in 図2.7.2 at P.121 in the textbook. 

以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/02. 

---
---
# A result on GC with GPU


<pre>
CPUs are ---
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz

GPU (if assigned) is ---
Sun Aug  1 12:10:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   9200MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|

<pre>
使用デバイス： cuda:0
-------------
Epoch 1/50
-------------
（train）
イテレーション 10 || Loss: 15.5047 || 10iter: 23.1471 sec.
イテレーション 20 || Loss: 13.1873 || 10iter: 15.0134 sec.
イテレーション 30 || Loss: 9.8160 || 10iter: 15.4017 sec.
イテレーション 40 || Loss: 9.0004 || 10iter: 15.6720 sec.
イテレーション 50 || Loss: 8.7081 || 10iter: 14.7855 sec.
イテレーション 60 || Loss: 7.9490 || 10iter: 15.7062 sec.
イテレーション 70 || Loss: 8.0755 || 10iter: 14.9899 sec.
イテレーション 80 || Loss: 8.0940 || 10iter: 15.1357 sec.
イテレーション 90 || Loss: 8.4861 || 10iter: 15.2473 sec.
イテレーション 100 || Loss: 8.5071 || 10iter: 15.7811 sec.
イテレーション 110 || Loss: 8.7733 || 10iter: 15.2426 sec.
イテレーション 120 || Loss: 8.2074 || 10iter: 15.2144 sec.
イテレーション 130 || Loss: 8.3611 || 10iter: 14.9032 sec.
イテレーション 140 || Loss: 8.1463 || 10iter: 15.5173 sec.
イテレーション 150 || Loss: 7.7141 || 10iter: 15.7221 sec.
イテレーション 160 || Loss: 8.4901 || 10iter: 16.3800 sec.
イテレーション 170 || Loss: 7.8081 || 10iter: 15.9001 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:1693.4835 ||Epoch_VAL_Loss:0.0000
timer:  300.5983 sec.
-------------
Epoch 2/50
-------------
（train）
イテレーション 180 || Loss: 8.2754 || 10iter: 1.1493 sec.
イテレーション 190 || Loss: 7.4293 || 10iter: 15.5905 sec.
イテレーション 200 || Loss: 7.4463 || 10iter: 15.5613 sec.
イテレーション 210 || Loss: 7.6427 || 10iter: 15.8041 sec.
イテレーション 220 || Loss: 7.0990 || 10iter: 16.4148 sec.
イテレーション 230 || Loss: 7.8312 || 10iter: 15.2834 sec.
イテレーション 240 || Loss: 7.7808 || 10iter: 15.8293 sec.
イテレーション 250 || Loss: 7.8077 || 10iter: 16.2850 sec.
イテレーション 260 || Loss: 7.2091 || 10iter: 15.5354 sec.
イテレーション 270 || Loss: 8.3113 || 10iter: 16.0509 sec.
イテレーション 280 || Loss: 7.4350 || 10iter: 15.7775 sec.
イテレーション 290 || Loss: 7.1210 || 10iter: 15.9230 sec.
イテレーション 300 || Loss: 7.1463 || 10iter: 15.1885 sec.
イテレーション 310 || Loss: 7.4287 || 10iter: 16.7222 sec.
イテレーション 320 || Loss: 6.9985 || 10iter: 16.2154 sec.
イテレーション 330 || Loss: 6.8802 || 10iter: 16.3296 sec.
イテレーション 340 || Loss: 6.9562 || 10iter: 15.4030 sec.
イテレーション 350 || Loss: 6.8974 || 10iter: 15.8240 sec.
-------------
epoch 2 || Epoch_TRAIN_Loss:1325.1945 ||Epoch_VAL_Loss:0.0000
timer:  295.3772 sec.
-------------
Epoch 3/50
-------------
（train）
イテレーション 360 || Loss: 6.8502 || 10iter: 2.6459 sec.
イテレーション 370 || Loss: 7.1720 || 10iter: 15.5002 sec.
イテレーション 380 || Loss: 7.4177 || 10iter: 15.7510 sec.
イテレーション 390 || Loss: 7.2238 || 10iter: 14.8070 sec.
イテレーション 400 || Loss: 6.8896 || 10iter: 15.9501 sec.
イテレーション 410 || Loss: 6.8275 || 10iter: 15.4774 sec.
イテレーション 420 || Loss: 6.8167 || 10iter: 15.8970 sec.
イテレーション 430 || Loss: 7.0659 || 10iter: 15.7212 sec.
イテレーション 440 || Loss: 6.8337 || 10iter: 15.2784 sec.
イテレーション 450 || Loss: 7.3559 || 10iter: 17.0530 sec.
イテレーション 460 || Loss: 7.8742 || 10iter: 15.5054 sec.
イテレーション 470 || Loss: 7.0422 || 10iter: 15.6777 sec.
イテレーション 480 || Loss: 6.6442 || 10iter: 15.5714 sec.
イテレーション 490 || Loss: 7.2784 || 10iter: 15.6204 sec.
イテレーション 500 || Loss: 7.2911 || 10iter: 16.2078 sec.
イテレーション 510 || Loss: 6.7506 || 10iter: 15.8633 sec.
イテレーション 520 || Loss: 6.3723 || 10iter: 16.4045 sec.
イテレーション 530 || Loss: 6.5858 || 10iter: 16.3512 sec.
-------------
epoch 3 || Epoch_TRAIN_Loss:1238.7102 ||Epoch_VAL_Loss:0.0000
timer:  294.5641 sec.
-------------
Epoch 4/50
-------------
（train）
イテレーション 540 || Loss: 6.9462 || 10iter: 4.5121 sec.
イテレーション 550 || Loss: 6.9330 || 10iter: 19.3549 sec.
イテレーション 560 || Loss: 6.8037 || 10iter: 16.6035 sec.
イテレーション 570 || Loss: 7.0172 || 10iter: 15.4525 sec.
イテレーション 580 || Loss: 6.9760 || 10iter: 15.7946 sec.
イテレーション 590 || Loss: 6.6603 || 10iter: 15.4716 sec.
イテレーション 600 || Loss: 6.9739 || 10iter: 14.8571 sec.
イテレーション 610 || Loss: 6.6289 || 10iter: 15.3234 sec.
イテレーション 620 || Loss: 6.3075 || 10iter: 15.8607 sec.
イテレーション 630 || Loss: 6.0115 || 10iter: 15.6214 sec.
イテレーション 640 || Loss: 6.7042 || 10iter: 15.6144 sec.
イテレーション 650 || Loss: 6.3086 || 10iter: 19.0593 sec.
イテレーション 660 || Loss: 6.4089 || 10iter: 15.2370 sec.
イテレーション 670 || Loss: 6.7724 || 10iter: 15.7041 sec.
イテレーション 680 || Loss: 6.1031 || 10iter: 15.9082 sec.
イテレーション 690 || Loss: 6.0135 || 10iter: 15.6284 sec.
イテレーション 700 || Loss: 6.0605 || 10iter: 15.1156 sec.
イテレーション 710 || Loss: 5.7897 || 10iter: 15.2593 sec.
-------------
epoch 4 || Epoch_TRAIN_Loss:1174.2462 ||Epoch_VAL_Loss:0.0000
timer:  297.4204 sec.
-------------
Epoch 5/50
-------------
（train）
イテレーション 720 || Loss: 5.8896 || 10iter: 5.7461 sec.
イテレーション 730 || Loss: 6.2937 || 10iter: 15.1301 sec.
イテレーション 740 || Loss: 6.1094 || 10iter: 15.8535 sec.
イテレーション 750 || Loss: 5.7158 || 10iter: 15.3296 sec.
イテレーション 760 || Loss: 5.9080 || 10iter: 16.0066 sec.
イテレーション 770 || Loss: 5.8113 || 10iter: 15.1264 sec.
イテレーション 780 || Loss: 6.3506 || 10iter: 15.4536 sec.
イテレーション 790 || Loss: 5.8635 || 10iter: 14.9777 sec.
イテレーション 800 || Loss: 6.2379 || 10iter: 16.2648 sec.
イテレーション 810 || Loss: 6.2692 || 10iter: 15.5452 sec.
イテレーション 820 || Loss: 6.1451 || 10iter: 15.7217 sec.
イテレーション 830 || Loss: 5.6888 || 10iter: 15.2199 sec.
イテレーション 840 || Loss: 5.9689 || 10iter: 15.3126 sec.
イテレーション 850 || Loss: 6.5327 || 10iter: 15.7915 sec.
イテレーション 860 || Loss: 6.3176 || 10iter: 19.2460 sec.
イテレーション 870 || Loss: 6.0963 || 10iter: 15.4634 sec.
イテレーション 880 || Loss: 6.2006 || 10iter: 14.8756 sec.
イテレーション 890 || Loss: 5.9343 || 10iter: 15.8041 sec.
-------------
epoch 5 || Epoch_TRAIN_Loss:1114.1584 ||Epoch_VAL_Loss:0.0000
timer:  292.1721 sec.
-------------
Epoch 6/50
-------------
（train）
イテレーション 900 || Loss: 6.1263 || 10iter: 7.4700 sec.
イテレーション 910 || Loss: 6.2481 || 10iter: 15.2408 sec.
イテレーション 920 || Loss: 6.1623 || 10iter: 14.7705 sec.
イテレーション 930 || Loss: 6.6335 || 10iter: 15.5062 sec.
イテレーション 940 || Loss: 5.6977 || 10iter: 14.9589 sec.
イテレーション 950 || Loss: 5.6356 || 10iter: 15.4387 sec.
イテレーション 960 || Loss: 6.2769 || 10iter: 15.0740 sec.
イテレーション 970 || Loss: 5.6971 || 10iter: 15.0899 sec.
イテレーション 980 || Loss: 6.0271 || 10iter: 15.7966 sec.
イテレーション 990 || Loss: 6.0332 || 10iter: 14.9973 sec.
イテレーション 1000 || Loss: 6.3409 || 10iter: 15.3521 sec.
イテレーション 1010 || Loss: 6.0453 || 10iter: 15.6639 sec.
イテレーション 1020 || Loss: 5.5742 || 10iter: 14.6871 sec.
イテレーション 1030 || Loss: 6.0900 || 10iter: 15.6594 sec.
イテレーション 1040 || Loss: 5.8397 || 10iter: 15.2977 sec.
イテレーション 1050 || Loss: 5.7209 || 10iter: 15.3414 sec.
イテレーション 1060 || Loss: 6.1257 || 10iter: 15.4536 sec.
イテレーション 1070 || Loss: 5.6638 || 10iter: 14.7330 sec.
-------------
epoch 6 || Epoch_TRAIN_Loss:1075.0307 ||Epoch_VAL_Loss:0.0000
timer:  284.1807 sec.
-------------
Epoch 7/50
-------------
（train）
イテレーション 1080 || Loss: 6.0761 || 10iter: 9.7507 sec.
イテレーション 1090 || Loss: 6.0854 || 10iter: 15.2327 sec.
イテレーション 1100 || Loss: 6.0582 || 10iter: 15.2899 sec.
イテレーション 1110 || Loss: 5.4615 || 10iter: 14.9114 sec.
イテレーション 1120 || Loss: 5.8669 || 10iter: 15.1400 sec.
イテレーション 1130 || Loss: 5.7162 || 10iter: 15.4532 sec.
イテレーション 1140 || Loss: 5.7785 || 10iter: 14.8632 sec.
イテレーション 1150 || Loss: 5.5613 || 10iter: 15.3757 sec.
イテレーション 1160 || Loss: 5.8620 || 10iter: 15.8652 sec.
イテレーション 1170 || Loss: 5.8936 || 10iter: 15.2189 sec.
イテレーション 1180 || Loss: 5.8820 || 10iter: 15.0387 sec.
イテレーション 1190 || Loss: 6.1864 || 10iter: 16.1979 sec.
イテレーション 1200 || Loss: 5.3948 || 10iter: 14.7456 sec.
イテレーション 1210 || Loss: 5.1589 || 10iter: 15.5536 sec.
イテレーション 1220 || Loss: 5.6994 || 10iter: 15.4056 sec.
イテレーション 1230 || Loss: 5.7105 || 10iter: 15.4974 sec.
イテレーション 1240 || Loss: 5.7697 || 10iter: 15.6403 sec.
イテレーション 1250 || Loss: 5.8949 || 10iter: 15.4005 sec.
-------------
epoch 7 || Epoch_TRAIN_Loss:1039.0141 ||Epoch_VAL_Loss:0.0000
timer:  286.6005 sec.
-------------
Epoch 8/50
-------------
（train）
イテレーション 1260 || Loss: 5.7505 || 10iter: 10.7684 sec.
イテレーション 1270 || Loss: 5.3228 || 10iter: 15.0144 sec.
イテレーション 1280 || Loss: 5.2932 || 10iter: 15.6378 sec.
イテレーション 1290 || Loss: 5.2180 || 10iter: 15.1416 sec.
イテレーション 1300 || Loss: 5.7232 || 10iter: 16.8353 sec.
イテレーション 1310 || Loss: 5.4252 || 10iter: 16.1316 sec.
イテレーション 1320 || Loss: 5.6980 || 10iter: 18.3036 sec.
イテレーション 1330 || Loss: 6.1320 || 10iter: 17.4915 sec.
イテレーション 1340 || Loss: 5.6854 || 10iter: 15.9135 sec.
イテレーション 1350 || Loss: 5.0712 || 10iter: 15.8852 sec.
イテレーション 1360 || Loss: 5.2988 || 10iter: 15.9137 sec.
イテレーション 1370 || Loss: 5.5523 || 10iter: 15.7579 sec.
イテレーション 1380 || Loss: 5.1331 || 10iter: 15.3882 sec.
イテレーション 1390 || Loss: 5.8975 || 10iter: 16.1099 sec.
イテレーション 1400 || Loss: 5.4924 || 10iter: 15.4859 sec.
イテレーション 1410 || Loss: 5.3270 || 10iter: 15.8471 sec.
イテレーション 1420 || Loss: 5.2382 || 10iter: 15.9550 sec.
イテレーション 1430 || Loss: 5.4124 || 10iter: 15.3322 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:1007.6194 ||Epoch_VAL_Loss:0.0000
timer:  297.2307 sec.
-------------
Epoch 9/50
-------------
（train）
イテレーション 1440 || Loss: 5.7330 || 10iter: 12.2765 sec.
イテレーション 1450 || Loss: 5.4616 || 10iter: 15.5604 sec.
イテレーション 1460 || Loss: 5.6357 || 10iter: 15.4636 sec.
イテレーション 1470 || Loss: 5.7660 || 10iter: 15.4817 sec.
イテレーション 1480 || Loss: 5.6430 || 10iter: 15.8120 sec.
イテレーション 1490 || Loss: 5.6630 || 10iter: 18.8742 sec.
イテレーション 1500 || Loss: 5.5823 || 10iter: 16.0314 sec.
イテレーション 1510 || Loss: 5.6966 || 10iter: 16.9232 sec.
イテレーション 1520 || Loss: 5.8412 || 10iter: 16.4896 sec.
イテレーション 1530 || Loss: 5.3578 || 10iter: 16.9793 sec.
イテレーション 1540 || Loss: 5.0490 || 10iter: 15.2927 sec.
イテレーション 1550 || Loss: 5.2917 || 10iter: 16.1113 sec.
イテレーション 1560 || Loss: 5.4565 || 10iter: 15.1301 sec.
イテレーション 1570 || Loss: 5.3115 || 10iter: 15.8667 sec.
イテレーション 1580 || Loss: 5.4828 || 10iter: 16.0564 sec.
イテレーション 1590 || Loss: 5.1524 || 10iter: 14.9224 sec.
イテレーション 1600 || Loss: 5.1428 || 10iter: 15.1491 sec.
イテレーション 1610 || Loss: 5.2094 || 10iter: 16.9746 sec.
-------------
epoch 9 || Epoch_TRAIN_Loss:968.5960 ||Epoch_VAL_Loss:0.0000
timer:  298.4585 sec.
-------------
Epoch 10/50
-------------
（train）
イテレーション 1620 || Loss: 5.8310 || 10iter: 15.8234 sec.
イテレーション 1630 || Loss: 5.3431 || 10iter: 16.1199 sec.
イテレーション 1640 || Loss: 5.2986 || 10iter: 15.1236 sec.
イテレーション 1650 || Loss: 5.1111 || 10iter: 16.0017 sec.
イテレーション 1660 || Loss: 5.9789 || 10iter: 15.2473 sec.
イテレーション 1670 || Loss: 5.2235 || 10iter: 15.5355 sec.
イテレーション 1680 || Loss: 4.4942 || 10iter: 15.6859 sec.
イテレーション 1690 || Loss: 5.6785 || 10iter: 16.7246 sec.
イテレーション 1700 || Loss: 5.0180 || 10iter: 16.4640 sec.
イテレーション 1710 || Loss: 5.6060 || 10iter: 16.1967 sec.
イテレーション 1720 || Loss: 5.1734 || 10iter: 15.5287 sec.
イテレーション 1730 || Loss: 5.4835 || 10iter: 15.2121 sec.
イテレーション 1740 || Loss: 4.8796 || 10iter: 15.2796 sec.
イテレーション 1750 || Loss: 5.1319 || 10iter: 19.6607 sec.
イテレーション 1760 || Loss: 5.5701 || 10iter: 15.3132 sec.
イテレーション 1770 || Loss: 5.4688 || 10iter: 15.0417 sec.
イテレーション 1780 || Loss: 5.4757 || 10iter: 17.9967 sec.
イテレーション 1790 || Loss: 4.7314 || 10iter: 16.1053 sec.
-------------
（val）
-------------
epoch 10 || Epoch_TRAIN_Loss:945.3035 ||Epoch_VAL_Loss:957.5100
timer:  422.2790 sec.
-------------
Epoch 11/50
-------------
（train）
イテレーション 1800 || Loss: 5.0329 || 10iter: 15.0038 sec.
イテレーション 1810 || Loss: 4.9723 || 10iter: 15.0667 sec.
イテレーション 1820 || Loss: 5.2915 || 10iter: 15.1830 sec.
イテレーション 1830 || Loss: 4.6034 || 10iter: 14.7423 sec.
イテレーション 1840 || Loss: 5.0721 || 10iter: 15.4249 sec.
イテレーション 1850 || Loss: 5.0130 || 10iter: 15.2954 sec.
イテレーション 1860 || Loss: 5.3338 || 10iter: 15.2149 sec.
イテレーション 1870 || Loss: 5.3757 || 10iter: 15.0966 sec.
イテレーション 1880 || Loss: 5.0654 || 10iter: 15.0662 sec.
イテレーション 1890 || Loss: 5.7380 || 10iter: 21.7720 sec.
イテレーション 1900 || Loss: 5.3690 || 10iter: 17.8880 sec.
イテレーション 1910 || Loss: 4.9345 || 10iter: 15.6364 sec.
イテレーション 1920 || Loss: 5.5199 || 10iter: 15.2800 sec.
イテレーション 1930 || Loss: 5.3061 || 10iter: 15.3852 sec.
イテレーション 1940 || Loss: 4.9942 || 10iter: 15.4572 sec.
イテレーション 1950 || Loss: 4.8753 || 10iter: 20.0238 sec.
イテレーション 1960 || Loss: 4.6045 || 10iter: 18.7790 sec.
-------------
epoch 11 || Epoch_TRAIN_Loss:933.0881 ||Epoch_VAL_Loss:0.0000
timer:  304.0331 sec.
-------------
Epoch 12/50
-------------
（train）
イテレーション 1970 || Loss: 5.6528 || 10iter: 1.3490 sec.
イテレーション 1980 || Loss: 4.9641 || 10iter: 16.6343 sec.
イテレーション 1990 || Loss: 5.4040 || 10iter: 16.7570 sec.
イテレーション 2000 || Loss: 5.2492 || 10iter: 15.0420 sec.
イテレーション 2010 || Loss: 4.4060 || 10iter: 16.1974 sec.
イテレーション 2020 || Loss: 4.3134 || 10iter: 16.2161 sec.
イテレーション 2030 || Loss: 5.0633 || 10iter: 15.2452 sec.
イテレーション 2040 || Loss: 4.9327 || 10iter: 16.7833 sec.
イテレーション 2050 || Loss: 5.0379 || 10iter: 14.8714 sec.
イテレーション 2060 || Loss: 5.3745 || 10iter: 16.5680 sec.
イテレーション 2070 || Loss: 5.4877 || 10iter: 16.6303 sec.
イテレーション 2080 || Loss: 5.3300 || 10iter: 15.0916 sec.
イテレーション 2090 || Loss: 5.6141 || 10iter: 14.9652 sec.
イテレーション 2100 || Loss: 4.7572 || 10iter: 15.3382 sec.
イテレーション 2110 || Loss: 4.7829 || 10iter: 15.2820 sec.
イテレーション 2120 || Loss: 4.8760 || 10iter: 15.0332 sec.
イテレーション 2130 || Loss: 5.2725 || 10iter: 15.2814 sec.
イテレーション 2140 || Loss: 4.9596 || 10iter: 15.0088 sec.
-------------
epoch 12 || Epoch_TRAIN_Loss:905.3060 ||Epoch_VAL_Loss:0.0000
timer:  292.0956 sec.
-------------
Epoch 13/50
-------------
（train）
イテレーション 2150 || Loss: 5.0765 || 10iter: 2.3561 sec.
イテレーション 2160 || Loss: 5.0783 || 10iter: 15.0770 sec.
イテレーション 2170 || Loss: 5.2190 || 10iter: 15.5530 sec.
イテレーション 2180 || Loss: 5.6792 || 10iter: 16.8319 sec.
イテレーション 2190 || Loss: 4.8920 || 10iter: 21.1149 sec.
イテレーション 2200 || Loss: 4.8519 || 10iter: 16.7671 sec.
イテレーション 2210 || Loss: 4.6354 || 10iter: 19.3837 sec.
イテレーション 2220 || Loss: 4.4611 || 10iter: 17.1006 sec.
イテレーション 2230 || Loss: 5.1971 || 10iter: 14.8162 sec.
イテレーション 2240 || Loss: 5.7194 || 10iter: 15.1728 sec.
イテレーション 2250 || Loss: 4.6536 || 10iter: 15.2423 sec.
イテレーション 2260 || Loss: 5.0995 || 10iter: 15.1281 sec.
イテレーション 2270 || Loss: 4.8263 || 10iter: 14.6782 sec.
イテレーション 2280 || Loss: 4.8091 || 10iter: 15.0932 sec.
イテレーション 2290 || Loss: 5.1603 || 10iter: 15.2206 sec.
イテレーション 2300 || Loss: 4.9417 || 10iter: 14.9200 sec.
イテレーション 2310 || Loss: 5.6429 || 10iter: 20.1280 sec.
イテレーション 2320 || Loss: 4.7823 || 10iter: 21.8087 sec.
-------------
epoch 13 || Epoch_TRAIN_Loss:887.3055 ||Epoch_VAL_Loss:0.0000
timer:  311.3158 sec.
-------------
Epoch 14/50
-------------
（train）
イテレーション 2330 || Loss: 4.7095 || 10iter: 4.2025 sec.
イテレーション 2340 || Loss: 4.3731 || 10iter: 16.5526 sec.
イテレーション 2350 || Loss: 5.3852 || 10iter: 15.9433 sec.
イテレーション 2360 || Loss: 5.1554 || 10iter: 14.6831 sec.
イテレーション 2370 || Loss: 5.0959 || 10iter: 14.8557 sec.
イテレーション 2380 || Loss: 5.2267 || 10iter: 15.1168 sec.
イテレーション 2390 || Loss: 3.8851 || 10iter: 14.7892 sec.
イテレーション 2400 || Loss: 4.5006 || 10iter: 14.7998 sec.
イテレーション 2410 || Loss: 4.9396 || 10iter: 14.8374 sec.
イテレーション 2420 || Loss: 4.9548 || 10iter: 15.0195 sec.
イテレーション 2430 || Loss: 4.9705 || 10iter: 14.9450 sec.
イテレーション 2440 || Loss: 5.1093 || 10iter: 14.8559 sec.
イテレーション 2450 || Loss: 5.3671 || 10iter: 14.5940 sec.
イテレーション 2460 || Loss: 5.0321 || 10iter: 15.3877 sec.
イテレーション 2470 || Loss: 4.8605 || 10iter: 15.0970 sec.
イテレーション 2480 || Loss: 4.9109 || 10iter: 14.7102 sec.
イテレーション 2490 || Loss: 4.6028 || 10iter: 14.8228 sec.
イテレーション 2500 || Loss: 5.2552 || 10iter: 15.0340 sec.
-------------
epoch 14 || Epoch_TRAIN_Loss:874.0533 ||Epoch_VAL_Loss:0.0000
timer:  281.1821 sec.
-------------
Epoch 15/50
-------------
（train）
イテレーション 2510 || Loss: 4.9878 || 10iter: 5.6601 sec.
イテレーション 2520 || Loss: 4.6213 || 10iter: 15.0315 sec.
イテレーション 2530 || Loss: 5.1961 || 10iter: 15.1484 sec.
イテレーション 2540 || Loss: 4.6540 || 10iter: 15.5326 sec.
イテレーション 2550 || Loss: 5.2071 || 10iter: 14.9484 sec.
イテレーション 2560 || Loss: 3.9397 || 10iter: 14.9241 sec.
イテレーション 2570 || Loss: 4.4013 || 10iter: 15.2454 sec.
イテレーション 2580 || Loss: 4.4634 || 10iter: 14.9889 sec.
イテレーション 2590 || Loss: 4.5855 || 10iter: 14.8074 sec.
イテレーション 2600 || Loss: 4.5721 || 10iter: 15.1805 sec.
イテレーション 2610 || Loss: 4.1337 || 10iter: 15.2820 sec.
イテレーション 2620 || Loss: 4.7359 || 10iter: 17.9817 sec.
イテレーション 2630 || Loss: 4.9158 || 10iter: 15.2525 sec.
イテレーション 2640 || Loss: 4.3237 || 10iter: 15.1307 sec.
イテレーション 2650 || Loss: 4.7668 || 10iter: 15.7384 sec.
イテレーション 2660 || Loss: 4.3872 || 10iter: 15.2031 sec.
イテレーション 2670 || Loss: 4.6877 || 10iter: 14.9726 sec.
イテレーション 2680 || Loss: 4.4534 || 10iter: 14.9950 sec.
-------------
epoch 15 || Epoch_TRAIN_Loss:859.2855 ||Epoch_VAL_Loss:0.0000
timer:  285.2470 sec.
-------------
Epoch 16/50
-------------
（train）
イテレーション 2690 || Loss: 4.5876 || 10iter: 7.4131 sec.
イテレーション 2700 || Loss: 4.8720 || 10iter: 15.3844 sec.
イテレーション 2710 || Loss: 4.9085 || 10iter: 15.2887 sec.
イテレーション 2720 || Loss: 4.1959 || 10iter: 15.2073 sec.
イテレーション 2730 || Loss: 4.7721 || 10iter: 15.7615 sec.
イテレーション 2740 || Loss: 4.3481 || 10iter: 15.9566 sec.
イテレーション 2750 || Loss: 4.6832 || 10iter: 18.7540 sec.
イテレーション 2760 || Loss: 4.3880 || 10iter: 18.6659 sec.
イテレーション 2770 || Loss: 5.3009 || 10iter: 16.2785 sec.
イテレーション 2780 || Loss: 4.6197 || 10iter: 16.6661 sec.
イテレーション 2790 || Loss: 4.9053 || 10iter: 15.7352 sec.
イテレーション 2800 || Loss: 5.0560 || 10iter: 16.2477 sec.
イテレーション 2810 || Loss: 4.5472 || 10iter: 15.7029 sec.
イテレーション 2820 || Loss: 4.8044 || 10iter: 14.8304 sec.
イテレーション 2830 || Loss: 4.5939 || 10iter: 15.2788 sec.
イテレーション 2840 || Loss: 4.4294 || 10iter: 15.3393 sec.
イテレーション 2850 || Loss: 4.8929 || 10iter: 14.9444 sec.
イテレーション 2860 || Loss: 4.6714 || 10iter: 15.2330 sec.
-------------
epoch 16 || Epoch_TRAIN_Loss:842.3203 ||Epoch_VAL_Loss:0.0000
timer:  296.2942 sec.
-------------
Epoch 17/50
-------------
（train）
イテレーション 2870 || Loss: 4.8329 || 10iter: 9.0420 sec.
イテレーション 2880 || Loss: 4.4427 || 10iter: 15.1923 sec.
イテレーション 2890 || Loss: 4.5396 || 10iter: 20.8405 sec.
イテレーション 2900 || Loss: 4.5915 || 10iter: 16.1849 sec.
イテレーション 2910 || Loss: 4.6450 || 10iter: 15.4808 sec.
イテレーション 2920 || Loss: 4.6368 || 10iter: 15.4819 sec.
イテレーション 2930 || Loss: 5.2062 || 10iter: 16.3556 sec.
イテレーション 2940 || Loss: 4.9268 || 10iter: 15.7239 sec.
イテレーション 2950 || Loss: 4.2530 || 10iter: 16.3152 sec.
イテレーション 2960 || Loss: 4.3235 || 10iter: 15.7433 sec.
イテレーション 2970 || Loss: 5.4032 || 10iter: 16.0881 sec.
イテレーション 2980 || Loss: 4.4743 || 10iter: 15.7047 sec.
イテレーション 2990 || Loss: 4.8811 || 10iter: 15.0933 sec.
イテレーション 3000 || Loss: 4.5524 || 10iter: 16.0088 sec.
イテレーション 3010 || Loss: 4.6891 || 10iter: 21.8298 sec.
イテレーション 3020 || Loss: 4.5238 || 10iter: 16.3612 sec.
イテレーション 3030 || Loss: 4.3906 || 10iter: 15.5449 sec.
イテレーション 3040 || Loss: 4.5854 || 10iter: 16.9163 sec.
-------------
epoch 17 || Epoch_TRAIN_Loss:832.8322 ||Epoch_VAL_Loss:0.0000
timer:  306.0818 sec.
-------------
Epoch 18/50
-------------
（train）
イテレーション 3050 || Loss: 4.2660 || 10iter: 11.2139 sec.
イテレーション 3060 || Loss: 4.4138 || 10iter: 18.9543 sec.
イテレーション 3070 || Loss: 4.4060 || 10iter: 16.5571 sec.
イテレーション 3080 || Loss: 4.3212 || 10iter: 17.3103 sec.
イテレーション 3090 || Loss: 4.7003 || 10iter: 15.7718 sec.
イテレーション 3100 || Loss: 4.1463 || 10iter: 15.4948 sec.
イテレーション 3110 || Loss: 4.0870 || 10iter: 16.9945 sec.
イテレーション 3120 || Loss: 4.3834 || 10iter: 14.8610 sec.
イテレーション 3130 || Loss: 4.8619 || 10iter: 15.0225 sec.
イテレーション 3140 || Loss: 4.4048 || 10iter: 17.4254 sec.
イテレーション 3150 || Loss: 4.7333 || 10iter: 15.1693 sec.
イテレーション 3160 || Loss: 4.4862 || 10iter: 15.0942 sec.
イテレーション 3170 || Loss: 4.7251 || 10iter: 14.7891 sec.
イテレーション 3180 || Loss: 4.4593 || 10iter: 15.2765 sec.
イテレーション 3190 || Loss: 4.7417 || 10iter: 15.1158 sec.
イテレーション 3200 || Loss: 4.0590 || 10iter: 14.6656 sec.
イテレーション 3210 || Loss: 4.4366 || 10iter: 15.7766 sec.
イテレーション 3220 || Loss: 4.5715 || 10iter: 20.2018 sec.
-------------
epoch 18 || Epoch_TRAIN_Loss:817.5189 ||Epoch_VAL_Loss:0.0000
timer:  301.0223 sec.
-------------
Epoch 19/50
-------------
（train）
イテレーション 3230 || Loss: 4.0475 || 10iter: 13.0784 sec.
イテレーション 3240 || Loss: 4.5463 || 10iter: 16.1844 sec.
イテレーション 3250 || Loss: 4.4468 || 10iter: 15.4105 sec.
イテレーション 3260 || Loss: 4.4029 || 10iter: 15.3480 sec.
イテレーション 3270 || Loss: 4.5062 || 10iter: 15.2538 sec.
イテレーション 3280 || Loss: 4.5326 || 10iter: 15.0401 sec.
イテレーション 3290 || Loss: 3.8451 || 10iter: 15.0809 sec.
イテレーション 3300 || Loss: 4.4274 || 10iter: 15.3946 sec.
イテレーション 3310 || Loss: 4.3292 || 10iter: 14.8849 sec.
イテレーション 3320 || Loss: 4.2237 || 10iter: 15.3166 sec.
イテレーション 3330 || Loss: 4.6131 || 10iter: 15.1179 sec.
イテレーション 3340 || Loss: 4.6696 || 10iter: 15.1461 sec.
イテレーション 3350 || Loss: 4.7338 || 10iter: 14.7285 sec.
イテレーション 3360 || Loss: 4.5250 || 10iter: 14.8202 sec.
イテレーション 3370 || Loss: 4.7328 || 10iter: 15.1997 sec.
イテレーション 3380 || Loss: 4.5507 || 10iter: 15.0207 sec.
イテレーション 3390 || Loss: 4.5576 || 10iter: 15.0139 sec.
イテレーション 3400 || Loss: 4.2897 || 10iter: 14.8942 sec.
-------------
epoch 19 || Epoch_TRAIN_Loss:805.1296 ||Epoch_VAL_Loss:0.0000
timer:  283.9016 sec.
-------------
Epoch 20/50
-------------
（train）
イテレーション 3410 || Loss: 4.8453 || 10iter: 14.0160 sec.
イテレーション 3420 || Loss: 4.7275 || 10iter: 14.9470 sec.
イテレーション 3430 || Loss: 4.3140 || 10iter: 14.6398 sec.
イテレーション 3440 || Loss: 4.7071 || 10iter: 15.0093 sec.
イテレーション 3450 || Loss: 4.6422 || 10iter: 15.1349 sec.
イテレーション 3460 || Loss: 4.0818 || 10iter: 15.1894 sec.
イテレーション 3470 || Loss: 4.1141 || 10iter: 15.0930 sec.
イテレーション 3480 || Loss: 4.2695 || 10iter: 14.8410 sec.
イテレーション 3490 || Loss: 4.4883 || 10iter: 15.3844 sec.
イテレーション 3500 || Loss: 4.4787 || 10iter: 15.2047 sec.
イテレーション 3510 || Loss: 3.8340 || 10iter: 14.9442 sec.
イテレーション 3520 || Loss: 4.1218 || 10iter: 15.1417 sec.
イテレーション 3530 || Loss: 4.2272 || 10iter: 14.5342 sec.
イテレーション 3540 || Loss: 3.8930 || 10iter: 14.9563 sec.
イテレーション 3550 || Loss: 4.2858 || 10iter: 14.7885 sec.
イテレーション 3560 || Loss: 4.5847 || 10iter: 15.2389 sec.
イテレーション 3570 || Loss: 4.5087 || 10iter: 14.9295 sec.
イテレーション 3580 || Loss: 4.1896 || 10iter: 14.5426 sec.
-------------
（val）
-------------
epoch 20 || Epoch_TRAIN_Loss:792.8190 ||Epoch_VAL_Loss:818.9669
timer:  389.3835 sec.
-------------
Epoch 21/50
-------------
（train）
イテレーション 3590 || Loss: 4.1115 || 10iter: 15.1557 sec.
イテレーション 3600 || Loss: 4.1968 || 10iter: 15.1489 sec.
イテレーション 3610 || Loss: 4.7951 || 10iter: 14.9507 sec.
イテレーション 3620 || Loss: 3.9387 || 10iter: 15.2454 sec.
イテレーション 3630 || Loss: 4.2033 || 10iter: 14.8542 sec.
イテレーション 3640 || Loss: 4.6038 || 10iter: 14.9690 sec.
イテレーション 3650 || Loss: 4.1043 || 10iter: 15.0733 sec.
イテレーション 3660 || Loss: 4.7910 || 10iter: 14.9411 sec.
イテレーション 3670 || Loss: 4.5738 || 10iter: 15.2464 sec.
イテレーション 3680 || Loss: 4.1808 || 10iter: 15.1768 sec.
イテレーション 3690 || Loss: 5.1178 || 10iter: 14.9762 sec.
イテレーション 3700 || Loss: 4.6308 || 10iter: 14.9036 sec.
イテレーション 3710 || Loss: 4.4073 || 10iter: 15.0689 sec.
イテレーション 3720 || Loss: 4.5152 || 10iter: 15.4966 sec.
イテレーション 3730 || Loss: 4.3075 || 10iter: 14.9997 sec.
イテレーション 3740 || Loss: 4.3917 || 10iter: 14.6203 sec.
イテレーション 3750 || Loss: 4.4567 || 10iter: 14.8506 sec.
-------------
epoch 21 || Epoch_TRAIN_Loss:783.6161 ||Epoch_VAL_Loss:0.0000
timer:  280.7112 sec.
-------------
Epoch 22/50
-------------
（train）
イテレーション 3760 || Loss: 4.5123 || 10iter: 1.0433 sec.
イテレーション 3770 || Loss: 4.6886 || 10iter: 15.0479 sec.
イテレーション 3780 || Loss: 4.1047 || 10iter: 14.9678 sec.
イテレーション 3790 || Loss: 3.9883 || 10iter: 14.8678 sec.
イテレーション 3800 || Loss: 4.0661 || 10iter: 15.1023 sec.
イテレーション 3810 || Loss: 4.3573 || 10iter: 15.2219 sec.
イテレーション 3820 || Loss: 4.2760 || 10iter: 15.1188 sec.
イテレーション 3830 || Loss: 4.6532 || 10iter: 14.5852 sec.
イテレーション 3840 || Loss: 4.5009 || 10iter: 15.0572 sec.
イテレーション 3850 || Loss: 4.5637 || 10iter: 15.3746 sec.
イテレーション 3860 || Loss: 4.3635 || 10iter: 14.9365 sec.
イテレーション 3870 || Loss: 4.2149 || 10iter: 15.5999 sec.
イテレーション 3880 || Loss: 3.9836 || 10iter: 14.6669 sec.
イテレーション 3890 || Loss: 4.5239 || 10iter: 15.1342 sec.
イテレーション 3900 || Loss: 4.0240 || 10iter: 14.9862 sec.
イテレーション 3910 || Loss: 3.8846 || 10iter: 15.1602 sec.
イテレーション 3920 || Loss: 3.8973 || 10iter: 14.6988 sec.
イテレーション 3930 || Loss: 3.8667 || 10iter: 15.0722 sec.
-------------
epoch 22 || Epoch_TRAIN_Loss:772.4140 ||Epoch_VAL_Loss:0.0000
timer:  280.4863 sec.
-------------
Epoch 23/50
-------------
（train）
イテレーション 3940 || Loss: 4.5497 || 10iter: 2.5024 sec.
イテレーション 3950 || Loss: 4.0412 || 10iter: 15.3410 sec.
イテレーション 3960 || Loss: 4.1691 || 10iter: 14.9129 sec.
イテレーション 3970 || Loss: 4.7092 || 10iter: 14.9853 sec.
イテレーション 3980 || Loss: 3.8351 || 10iter: 14.9741 sec.
イテレーション 3990 || Loss: 3.9122 || 10iter: 15.2728 sec.
イテレーション 4000 || Loss: 4.0808 || 10iter: 14.9897 sec.
イテレーション 4010 || Loss: 4.0239 || 10iter: 14.9161 sec.
イテレーション 4020 || Loss: 4.2901 || 10iter: 15.0728 sec.
イテレーション 4030 || Loss: 4.4749 || 10iter: 15.1577 sec.
イテレーション 4040 || Loss: 4.8709 || 10iter: 15.3571 sec.
イテレーション 4050 || Loss: 4.5085 || 10iter: 14.9342 sec.
イテレーション 4060 || Loss: 4.4301 || 10iter: 14.8753 sec.
イテレーション 4070 || Loss: 3.9311 || 10iter: 14.8630 sec.
イテレーション 4080 || Loss: 3.8352 || 10iter: 15.5844 sec.
イテレーション 4090 || Loss: 3.8724 || 10iter: 15.0778 sec.
イテレーション 4100 || Loss: 4.1801 || 10iter: 15.2706 sec.
イテレーション 4110 || Loss: 4.1871 || 10iter: 14.9298 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:765.5067 ||Epoch_VAL_Loss:0.0000
timer:  281.3767 sec.
-------------
Epoch 24/50
-------------
（train）
イテレーション 4120 || Loss: 4.5733 || 10iter: 4.1652 sec.
イテレーション 4130 || Loss: 4.2683 || 10iter: 14.8492 sec.
イテレーション 4140 || Loss: 4.2416 || 10iter: 15.3090 sec.
イテレーション 4150 || Loss: 4.2727 || 10iter: 15.0893 sec.
イテレーション 4160 || Loss: 3.9301 || 10iter: 14.6562 sec.
イテレーション 4170 || Loss: 3.7577 || 10iter: 14.7689 sec.
イテレーション 4180 || Loss: 3.6696 || 10iter: 15.3444 sec.
イテレーション 4190 || Loss: 4.4610 || 10iter: 15.2596 sec.
イテレーション 4200 || Loss: 4.1795 || 10iter: 15.0435 sec.
イテレーション 4210 || Loss: 4.7465 || 10iter: 14.8943 sec.
イテレーション 4220 || Loss: 4.7248 || 10iter: 14.9734 sec.
イテレーション 4230 || Loss: 3.7360 || 10iter: 15.2009 sec.
イテレーション 4240 || Loss: 4.5879 || 10iter: 14.7941 sec.
イテレーション 4250 || Loss: 4.5565 || 10iter: 14.6732 sec.
イテレーション 4260 || Loss: 3.9297 || 10iter: 14.9768 sec.
イテレーション 4270 || Loss: 4.4437 || 10iter: 15.2859 sec.
イテレーション 4280 || Loss: 3.8715 || 10iter: 15.1790 sec.
イテレーション 4290 || Loss: 4.4798 || 10iter: 15.1933 sec.
-------------
epoch 24 || Epoch_TRAIN_Loss:753.1351 ||Epoch_VAL_Loss:0.0000
timer:  280.3923 sec.
-------------
Epoch 25/50
-------------
（train）
イテレーション 4300 || Loss: 3.7973 || 10iter: 5.6358 sec.
イテレーション 4310 || Loss: 4.0337 || 10iter: 14.8720 sec.
イテレーション 4320 || Loss: 3.6882 || 10iter: 14.7905 sec.
イテレーション 4330 || Loss: 4.0254 || 10iter: 15.0555 sec.
イテレーション 4340 || Loss: 3.7526 || 10iter: 15.0392 sec.
イテレーション 4350 || Loss: 4.0313 || 10iter: 14.8227 sec.
イテレーション 4360 || Loss: 4.1421 || 10iter: 14.9617 sec.
イテレーション 4370 || Loss: 4.0471 || 10iter: 15.2552 sec.
イテレーション 4380 || Loss: 4.5164 || 10iter: 15.1004 sec.
イテレーション 4390 || Loss: 4.5490 || 10iter: 14.9700 sec.
イテレーション 4400 || Loss: 4.7864 || 10iter: 14.7993 sec.
イテレーション 4410 || Loss: 4.4220 || 10iter: 15.0402 sec.
イテレーション 4420 || Loss: 3.5507 || 10iter: 14.9100 sec.
イテレーション 4430 || Loss: 4.2466 || 10iter: 14.8497 sec.
イテレーション 4440 || Loss: 4.3234 || 10iter: 14.6530 sec.
イテレーション 4450 || Loss: 3.9516 || 10iter: 15.1355 sec.
イテレーション 4460 || Loss: 3.5770 || 10iter: 15.2321 sec.
イテレーション 4470 || Loss: 4.4528 || 10iter: 14.8433 sec.
-------------
epoch 25 || Epoch_TRAIN_Loss:745.5489 ||Epoch_VAL_Loss:0.0000
timer:  279.1493 sec.
-------------
Epoch 26/50
-------------
（train）
イテレーション 4480 || Loss: 4.2871 || 10iter: 7.1476 sec.
イテレーション 4490 || Loss: 3.8243 || 10iter: 14.9989 sec.
イテレーション 4500 || Loss: 4.1329 || 10iter: 15.5342 sec.
イテレーション 4510 || Loss: 4.1289 || 10iter: 14.8567 sec.
イテレーション 4520 || Loss: 4.1073 || 10iter: 15.3188 sec.
イテレーション 4530 || Loss: 4.3011 || 10iter: 15.2089 sec.
イテレーション 4540 || Loss: 3.9418 || 10iter: 15.0583 sec.
イテレーション 4550 || Loss: 4.1607 || 10iter: 14.7277 sec.
イテレーション 4560 || Loss: 4.2349 || 10iter: 15.1259 sec.
イテレーション 4570 || Loss: 4.3238 || 10iter: 15.0008 sec.
イテレーション 4580 || Loss: 4.3902 || 10iter: 14.5939 sec.
イテレーション 4590 || Loss: 4.0513 || 10iter: 15.1147 sec.
イテレーション 4600 || Loss: 4.0815 || 10iter: 15.1599 sec.
イテレーション 4610 || Loss: 3.8583 || 10iter: 15.2794 sec.
イテレーション 4620 || Loss: 3.6919 || 10iter: 14.6181 sec.
イテレーション 4630 || Loss: 3.8912 || 10iter: 14.8514 sec.
イテレーション 4640 || Loss: 4.0725 || 10iter: 15.1512 sec.
イテレーション 4650 || Loss: 4.2906 || 10iter: 15.5580 sec.
-------------
epoch 26 || Epoch_TRAIN_Loss:736.6890 ||Epoch_VAL_Loss:0.0000
timer:  280.8750 sec.
-------------
Epoch 27/50
-------------
（train）
イテレーション 4660 || Loss: 3.7296 || 10iter: 8.6713 sec.
イテレーション 4670 || Loss: 4.4508 || 10iter: 14.9987 sec.
イテレーション 4680 || Loss: 4.4241 || 10iter: 14.7812 sec.
イテレーション 4690 || Loss: 4.0746 || 10iter: 15.4717 sec.
イテレーション 4700 || Loss: 4.1979 || 10iter: 15.1645 sec.
イテレーション 4710 || Loss: 4.1811 || 10iter: 15.2606 sec.
イテレーション 4720 || Loss: 3.8400 || 10iter: 15.0761 sec.
イテレーション 4730 || Loss: 4.0734 || 10iter: 14.7592 sec.
イテレーション 4740 || Loss: 4.0779 || 10iter: 14.9794 sec.
イテレーション 4750 || Loss: 4.2961 || 10iter: 15.7060 sec.
イテレーション 4760 || Loss: 4.0960 || 10iter: 15.2812 sec.
イテレーション 4770 || Loss: 3.6137 || 10iter: 15.3143 sec.
イテレーション 4780 || Loss: 3.9692 || 10iter: 14.8824 sec.
イテレーション 4790 || Loss: 4.5539 || 10iter: 16.0120 sec.
イテレーション 4800 || Loss: 3.7059 || 10iter: 15.0443 sec.
イテレーション 4810 || Loss: 3.6060 || 10iter: 15.7639 sec.
イテレーション 4820 || Loss: 3.7043 || 10iter: 15.0263 sec.
イテレーション 4830 || Loss: 3.8654 || 10iter: 16.6343 sec.
-------------
epoch 27 || Epoch_TRAIN_Loss:733.0914 ||Epoch_VAL_Loss:0.0000
timer:  284.9664 sec.
-------------
Epoch 28/50
-------------
（train）
イテレーション 4840 || Loss: 3.9269 || 10iter: 10.4382 sec.
イテレーション 4850 || Loss: 3.6595 || 10iter: 15.7937 sec.
イテレーション 4860 || Loss: 3.7200 || 10iter: 14.6596 sec.
イテレーション 4870 || Loss: 3.7221 || 10iter: 16.9403 sec.
イテレーション 4880 || Loss: 4.1954 || 10iter: 15.2262 sec.
イテレーション 4890 || Loss: 4.3286 || 10iter: 16.3291 sec.
イテレーション 4900 || Loss: 3.9250 || 10iter: 14.8800 sec.
イテレーション 4910 || Loss: 3.8867 || 10iter: 16.0466 sec.
イテレーション 4920 || Loss: 4.2931 || 10iter: 16.6497 sec.
イテレーション 4930 || Loss: 3.8701 || 10iter: 15.0455 sec.
イテレーション 4940 || Loss: 4.1839 || 10iter: 16.6008 sec.
イテレーション 4950 || Loss: 4.7841 || 10iter: 14.9418 sec.
イテレーション 4960 || Loss: 3.9475 || 10iter: 16.8916 sec.
イテレーション 4970 || Loss: 4.0134 || 10iter: 15.5241 sec.
イテレーション 4980 || Loss: 3.9786 || 10iter: 15.3145 sec.
イテレーション 4990 || Loss: 3.6318 || 10iter: 16.7156 sec.
イテレーション 5000 || Loss: 4.5423 || 10iter: 15.1925 sec.
イテレーション 5010 || Loss: 4.1273 || 10iter: 15.9973 sec.
-------------
epoch 28 || Epoch_TRAIN_Loss:718.6153 ||Epoch_VAL_Loss:0.0000
timer:  294.4202 sec.
-------------
Epoch 29/50
-------------
（train）
イテレーション 5020 || Loss: 3.6917 || 10iter: 12.2138 sec.
イテレーション 5030 || Loss: 4.1376 || 10iter: 15.2630 sec.
イテレーション 5040 || Loss: 4.2088 || 10iter: 16.9410 sec.
イテレーション 5050 || Loss: 4.5089 || 10iter: 15.3048 sec.
イテレーション 5060 || Loss: 3.8585 || 10iter: 16.0234 sec.
イテレーション 5070 || Loss: 4.0874 || 10iter: 15.1198 sec.
イテレーション 5080 || Loss: 3.7813 || 10iter: 15.0199 sec.
イテレーション 5090 || Loss: 3.6712 || 10iter: 15.8003 sec.
イテレーション 5100 || Loss: 4.5464 || 10iter: 16.9846 sec.
イテレーション 5110 || Loss: 4.5023 || 10iter: 15.8153 sec.
イテレーション 5120 || Loss: 3.9668 || 10iter: 15.3096 sec.
イテレーション 5130 || Loss: 3.5433 || 10iter: 16.9823 sec.
イテレーション 5140 || Loss: 3.7653 || 10iter: 15.4409 sec.
イテレーション 5150 || Loss: 3.6967 || 10iter: 15.7949 sec.
イテレーション 5160 || Loss: 4.3367 || 10iter: 16.2191 sec.
イテレーション 5170 || Loss: 4.0088 || 10iter: 15.0953 sec.
イテレーション 5180 || Loss: 3.7425 || 10iter: 16.5629 sec.
イテレーション 5190 || Loss: 3.7365 || 10iter: 15.9020 sec.
-------------
epoch 29 || Epoch_TRAIN_Loss:718.3516 ||Epoch_VAL_Loss:0.0000
timer:  294.6855 sec.
-------------
Epoch 30/50
-------------
（train）
イテレーション 5200 || Loss: 4.5828 || 10iter: 13.7595 sec.
イテレーション 5210 || Loss: 4.0003 || 10iter: 16.8121 sec.
イテレーション 5220 || Loss: 3.8479 || 10iter: 15.2427 sec.
イテレーション 5230 || Loss: 3.8999 || 10iter: 16.9618 sec.
イテレーション 5240 || Loss: 3.9337 || 10iter: 15.9281 sec.
イテレーション 5250 || Loss: 3.8439 || 10iter: 14.7971 sec.
イテレーション 5260 || Loss: 3.7963 || 10iter: 17.4295 sec.
イテレーション 5270 || Loss: 3.8920 || 10iter: 15.6269 sec.
イテレーション 5280 || Loss: 3.8671 || 10iter: 15.8086 sec.
イテレーション 5290 || Loss: 3.9216 || 10iter: 16.4210 sec.
イテレーション 5300 || Loss: 4.1243 || 10iter: 14.9803 sec.
イテレーション 5310 || Loss: 4.3088 || 10iter: 14.8131 sec.
イテレーション 5320 || Loss: 3.6174 || 10iter: 17.9448 sec.
イテレーション 5330 || Loss: 4.0103 || 10iter: 16.5845 sec.
イテレーション 5340 || Loss: 4.0160 || 10iter: 15.2789 sec.
イテレーション 5350 || Loss: 3.3044 || 10iter: 14.9700 sec.
イテレーション 5360 || Loss: 4.3744 || 10iter: 15.2791 sec.
イテレーション 5370 || Loss: 5.4202 || 10iter: 15.2192 sec.
-------------
（val）
-------------
epoch 30 || Epoch_TRAIN_Loss:701.7615 ||Epoch_VAL_Loss:764.5722
timer:  401.8549 sec.
-------------
Epoch 31/50
-------------
（train）
イテレーション 5380 || Loss: 3.6364 || 10iter: 15.3186 sec.
イテレーション 5390 || Loss: 3.5507 || 10iter: 15.4175 sec.
イテレーション 5400 || Loss: 2.9872 || 10iter: 14.9556 sec.
イテレーション 5410 || Loss: 3.5861 || 10iter: 14.8898 sec.
イテレーション 5420 || Loss: 4.2079 || 10iter: 15.0774 sec.
イテレーション 5430 || Loss: 4.1191 || 10iter: 15.1854 sec.
イテレーション 5440 || Loss: 4.0086 || 10iter: 15.2206 sec.
イテレーション 5450 || Loss: 3.8697 || 10iter: 15.6716 sec.
イテレーション 5460 || Loss: 4.9019 || 10iter: 15.0771 sec.
イテレーション 5470 || Loss: 3.9150 || 10iter: 14.5890 sec.
イテレーション 5480 || Loss: 3.5631 || 10iter: 14.5140 sec.
イテレーション 5490 || Loss: 3.7168 || 10iter: 15.5283 sec.
イテレーション 5500 || Loss: 3.8499 || 10iter: 15.3219 sec.
イテレーション 5510 || Loss: 3.6713 || 10iter: 14.9667 sec.
イテレーション 5520 || Loss: 4.0100 || 10iter: 15.0507 sec.
イテレーション 5530 || Loss: 3.6864 || 10iter: 14.8074 sec.
イテレーション 5540 || Loss: 3.9236 || 10iter: 15.1691 sec.
-------------
epoch 31 || Epoch_TRAIN_Loss:699.6601 ||Epoch_VAL_Loss:0.0000
timer:  281.5221 sec.
-------------
Epoch 32/50
-------------
（train）
イテレーション 5550 || Loss: 3.9531 || 10iter: 0.8593 sec.
イテレーション 5560 || Loss: 3.6340 || 10iter: 14.9275 sec.
イテレーション 5570 || Loss: 3.5687 || 10iter: 15.1372 sec.
イテレーション 5580 || Loss: 3.9642 || 10iter: 15.1169 sec.
イテレーション 5590 || Loss: 3.8927 || 10iter: 14.8751 sec.
イテレーション 5600 || Loss: 4.0696 || 10iter: 14.8968 sec.
イテレーション 5610 || Loss: 3.9934 || 10iter: 14.7074 sec.
イテレーション 5620 || Loss: 3.2996 || 10iter: 15.5060 sec.
イテレーション 5630 || Loss: 3.8007 || 10iter: 15.5167 sec.
イテレーション 5640 || Loss: 3.5975 || 10iter: 15.0972 sec.
イテレーション 5650 || Loss: 3.7025 || 10iter: 15.1197 sec.
イテレーション 5660 || Loss: 4.2169 || 10iter: 15.1049 sec.
イテレーション 5670 || Loss: 3.5008 || 10iter: 15.0550 sec.
イテレーション 5680 || Loss: 3.9486 || 10iter: 15.3768 sec.
イテレーション 5690 || Loss: 4.3655 || 10iter: 15.1433 sec.
イテレーション 5700 || Loss: 3.3791 || 10iter: 14.8593 sec.
イテレーション 5710 || Loss: 4.0547 || 10iter: 14.8000 sec.
イテレーション 5720 || Loss: 4.1127 || 10iter: 15.2457 sec.
-------------
epoch 32 || Epoch_TRAIN_Loss:694.2265 ||Epoch_VAL_Loss:0.0000
timer:  281.3741 sec.
-------------
Epoch 33/50
-------------
（train）
イテレーション 5730 || Loss: 3.8865 || 10iter: 2.4896 sec.
イテレーション 5740 || Loss: 3.9988 || 10iter: 15.0433 sec.
イテレーション 5750 || Loss: 3.7581 || 10iter: 14.8489 sec.
イテレーション 5760 || Loss: 3.5109 || 10iter: 15.4612 sec.
イテレーション 5770 || Loss: 3.9185 || 10iter: 15.3474 sec.
イテレーション 5780 || Loss: 4.2519 || 10iter: 14.8957 sec.
イテレーション 5790 || Loss: 3.2731 || 10iter: 14.8354 sec.
イテレーション 5800 || Loss: 3.9351 || 10iter: 15.1393 sec.
イテレーション 5810 || Loss: 3.9227 || 10iter: 15.4322 sec.
イテレーション 5820 || Loss: 3.5444 || 10iter: 14.8829 sec.
イテレーション 5830 || Loss: 4.3456 || 10iter: 14.9585 sec.
イテレーション 5840 || Loss: 3.9069 || 10iter: 15.0742 sec.
イテレーション 5850 || Loss: 3.4414 || 10iter: 15.2197 sec.
イテレーション 5860 || Loss: 4.0799 || 10iter: 15.3041 sec.
イテレーション 5870 || Loss: 3.7707 || 10iter: 15.2344 sec.
イテレーション 5880 || Loss: 3.5367 || 10iter: 15.0102 sec.
イテレーション 5890 || Loss: 3.3999 || 10iter: 15.0859 sec.
イテレーション 5900 || Loss: 3.3578 || 10iter: 14.9262 sec.
-------------
epoch 33 || Epoch_TRAIN_Loss:692.4667 ||Epoch_VAL_Loss:0.0000
timer:  281.8904 sec.
-------------
Epoch 34/50
-------------
（train）
イテレーション 5910 || Loss: 3.9519 || 10iter: 4.1219 sec.
イテレーション 5920 || Loss: 4.0296 || 10iter: 15.0238 sec.
イテレーション 5930 || Loss: 4.1519 || 10iter: 15.0294 sec.
イテレーション 5940 || Loss: 3.9412 || 10iter: 14.9253 sec.
イテレーション 5950 || Loss: 3.9449 || 10iter: 15.1937 sec.
イテレーション 5960 || Loss: 3.7930 || 10iter: 15.0759 sec.
イテレーション 5970 || Loss: 3.1981 || 10iter: 14.8676 sec.
イテレーション 5980 || Loss: 3.8363 || 10iter: 14.9133 sec.
イテレーション 5990 || Loss: 3.7814 || 10iter: 15.2127 sec.
イテレーション 6000 || Loss: 4.4401 || 10iter: 15.1962 sec.
イテレーション 6010 || Loss: 3.9440 || 10iter: 14.9786 sec.
イテレーション 6020 || Loss: 4.1437 || 10iter: 15.1910 sec.
イテレーション 6030 || Loss: 4.4591 || 10iter: 15.2929 sec.
イテレーション 6040 || Loss: 3.9365 || 10iter: 15.3691 sec.
イテレーション 6050 || Loss: 3.8340 || 10iter: 15.0020 sec.
イテレーション 6060 || Loss: 3.6671 || 10iter: 15.5121 sec.
イテレーション 6070 || Loss: 3.8245 || 10iter: 15.0368 sec.
イテレーション 6080 || Loss: 3.3396 || 10iter: 14.8286 sec.
-------------
epoch 34 || Epoch_TRAIN_Loss:684.6138 ||Epoch_VAL_Loss:0.0000
timer:  281.6169 sec.
-------------
Epoch 35/50
-------------
（train）
イテレーション 6090 || Loss: 3.9432 || 10iter: 5.6164 sec.
イテレーション 6100 || Loss: 4.1281 || 10iter: 15.6517 sec.
イテレーション 6110 || Loss: 3.7473 || 10iter: 15.2658 sec.
イテレーション 6120 || Loss: 3.7147 || 10iter: 14.6774 sec.
イテレーション 6130 || Loss: 3.8887 || 10iter: 14.9305 sec.
イテレーション 6140 || Loss: 3.8804 || 10iter: 15.5245 sec.
イテレーション 6150 || Loss: 3.7771 || 10iter: 15.0642 sec.
イテレーション 6160 || Loss: 3.8718 || 10iter: 14.7162 sec.
イテレーション 6170 || Loss: 3.9265 || 10iter: 14.9056 sec.
イテレーション 6180 || Loss: 4.0326 || 10iter: 15.1523 sec.
イテレーション 6190 || Loss: 3.6548 || 10iter: 15.3902 sec.
イテレーション 6200 || Loss: 3.9882 || 10iter: 14.8140 sec.
イテレーション 6210 || Loss: 4.3057 || 10iter: 15.1905 sec.
イテレーション 6220 || Loss: 3.9840 || 10iter: 15.4127 sec.
イテレーション 6230 || Loss: 3.9271 || 10iter: 15.3317 sec.
イテレーション 6240 || Loss: 3.4084 || 10iter: 14.8339 sec.
イテレーション 6250 || Loss: 3.9611 || 10iter: 14.8947 sec.
イテレーション 6260 || Loss: 3.3973 || 10iter: 15.0230 sec.
-------------
epoch 35 || Epoch_TRAIN_Loss:680.1697 ||Epoch_VAL_Loss:0.0000
timer:  281.7305 sec.
-------------
Epoch 36/50
-------------
（train）
イテレーション 6270 || Loss: 3.8240 || 10iter: 7.2481 sec.
イテレーション 6280 || Loss: 3.2370 || 10iter: 15.2745 sec.
イテレーション 6290 || Loss: 4.2176 || 10iter: 15.1416 sec.
イテレーション 6300 || Loss: 4.5644 || 10iter: 14.9186 sec.
イテレーション 6310 || Loss: 3.4733 || 10iter: 14.9997 sec.
イテレーション 6320 || Loss: 3.8429 || 10iter: 14.9645 sec.
イテレーション 6330 || Loss: 3.7863 || 10iter: 15.5117 sec.
イテレーション 6340 || Loss: 3.9805 || 10iter: 15.1547 sec.
イテレーション 6350 || Loss: 3.5477 || 10iter: 15.0856 sec.
イテレーション 6360 || Loss: 4.0964 || 10iter: 14.6789 sec.
イテレーション 6370 || Loss: 3.6822 || 10iter: 14.9401 sec.
イテレーション 6380 || Loss: 3.9540 || 10iter: 15.0803 sec.
イテレーション 6390 || Loss: 3.5185 || 10iter: 14.7105 sec.
イテレーション 6400 || Loss: 4.1206 || 10iter: 14.9667 sec.
イテレーション 6410 || Loss: 4.0875 || 10iter: 15.1303 sec.
イテレーション 6420 || Loss: 3.8998 || 10iter: 15.2856 sec.
イテレーション 6430 || Loss: 3.7844 || 10iter: 14.8519 sec.
イテレーション 6440 || Loss: 3.7043 || 10iter: 14.8655 sec.
-------------
epoch 36 || Epoch_TRAIN_Loss:677.1195 ||Epoch_VAL_Loss:0.0000
timer:  280.4269 sec.
-------------
Epoch 37/50
-------------
（train）
イテレーション 6450 || Loss: 3.7051 || 10iter: 8.6969 sec.
イテレーション 6460 || Loss: 4.0198 || 10iter: 15.0223 sec.
イテレーション 6470 || Loss: 3.1003 || 10iter: 15.2904 sec.
イテレーション 6480 || Loss: 3.4570 || 10iter: 15.3651 sec.
イテレーション 6490 || Loss: 3.7493 || 10iter: 15.2549 sec.
イテレーション 6500 || Loss: 3.6364 || 10iter: 15.0076 sec.
イテレーション 6510 || Loss: 3.9750 || 10iter: 15.1545 sec.
イテレーション 6520 || Loss: 3.8996 || 10iter: 15.0858 sec.
イテレーション 6530 || Loss: 3.7862 || 10iter: 14.9726 sec.
イテレーション 6540 || Loss: 3.4941 || 10iter: 14.7353 sec.
イテレーション 6550 || Loss: 3.6134 || 10iter: 14.8926 sec.
イテレーション 6560 || Loss: 3.7567 || 10iter: 15.1662 sec.
イテレーション 6570 || Loss: 3.8611 || 10iter: 15.4157 sec.
イテレーション 6580 || Loss: 3.9067 || 10iter: 14.7966 sec.
イテレーション 6590 || Loss: 4.0250 || 10iter: 14.8700 sec.
イテレーション 6600 || Loss: 3.4405 || 10iter: 15.1437 sec.
イテレーション 6610 || Loss: 3.9900 || 10iter: 15.4921 sec.
イテレーション 6620 || Loss: 3.7464 || 10iter: 14.9681 sec.
-------------
epoch 37 || Epoch_TRAIN_Loss:668.9858 ||Epoch_VAL_Loss:0.0000
timer:  281.4652 sec.
-------------
Epoch 38/50
-------------
（train）
イテレーション 6630 || Loss: 3.8784 || 10iter: 10.3787 sec.
イテレーション 6640 || Loss: 3.7145 || 10iter: 15.2406 sec.
イテレーション 6650 || Loss: 3.9291 || 10iter: 15.2913 sec.
イテレーション 6660 || Loss: 4.1714 || 10iter: 15.2243 sec.
イテレーション 6670 || Loss: 3.8068 || 10iter: 14.7834 sec.
イテレーション 6680 || Loss: 3.6307 || 10iter: 14.8011 sec.
イテレーション 6690 || Loss: 3.3128 || 10iter: 15.4286 sec.
イテレーション 6700 || Loss: 4.3242 || 10iter: 15.0838 sec.
イテレーション 6710 || Loss: 3.7150 || 10iter: 15.1565 sec.
イテレーション 6720 || Loss: 4.2567 || 10iter: 14.8155 sec.
イテレーション 6730 || Loss: 3.7062 || 10iter: 14.8843 sec.
イテレーション 6740 || Loss: 3.7504 || 10iter: 14.9597 sec.
イテレーション 6750 || Loss: 3.5913 || 10iter: 15.4044 sec.
イテレーション 6760 || Loss: 3.5228 || 10iter: 14.9596 sec.
イテレーション 6770 || Loss: 3.2848 || 10iter: 14.7921 sec.
イテレーション 6780 || Loss: 3.6444 || 10iter: 15.1974 sec.
イテレーション 6790 || Loss: 2.9087 || 10iter: 15.2917 sec.
イテレーション 6800 || Loss: 4.1859 || 10iter: 15.1760 sec.
-------------
epoch 38 || Epoch_TRAIN_Loss:660.5475 ||Epoch_VAL_Loss:0.0000
timer:  281.4427 sec.
-------------
Epoch 39/50
-------------
（train）
イテレーション 6810 || Loss: 3.5618 || 10iter: 11.6612 sec.
イテレーション 6820 || Loss: 3.5812 || 10iter: 14.8242 sec.
イテレーション 6830 || Loss: 4.1484 || 10iter: 15.1580 sec.
イテレーション 6840 || Loss: 3.4831 || 10iter: 14.9917 sec.
イテレーション 6850 || Loss: 3.6197 || 10iter: 14.9264 sec.
イテレーション 6860 || Loss: 3.7583 || 10iter: 15.0419 sec.
イテレーション 6870 || Loss: 3.9547 || 10iter: 15.0974 sec.
イテレーション 6880 || Loss: 3.5837 || 10iter: 15.3172 sec.
イテレーション 6890 || Loss: 3.8971 || 10iter: 15.0597 sec.
イテレーション 6900 || Loss: 3.2500 || 10iter: 15.2036 sec.
イテレーション 6910 || Loss: 3.3389 || 10iter: 14.8511 sec.
イテレーション 6920 || Loss: 3.1914 || 10iter: 14.9534 sec.
イテレーション 6930 || Loss: 3.5621 || 10iter: 14.7877 sec.
イテレーション 6940 || Loss: 3.9633 || 10iter: 15.2052 sec.
イテレーション 6950 || Loss: 4.1346 || 10iter: 14.9289 sec.
イテレーション 6960 || Loss: 3.5826 || 10iter: 15.1191 sec.
イテレーション 6970 || Loss: 3.3417 || 10iter: 15.3682 sec.
イテレーション 6980 || Loss: 4.0775 || 10iter: 15.0792 sec.
-------------
epoch 39 || Epoch_TRAIN_Loss:658.3879 ||Epoch_VAL_Loss:0.0000
timer:  280.6264 sec.
-------------
Epoch 40/50
-------------
（train）
イテレーション 6990 || Loss: 3.6537 || 10iter: 14.7010 sec.
イテレーション 7000 || Loss: 3.6628 || 10iter: 15.0722 sec.
イテレーション 7010 || Loss: 3.2654 || 10iter: 15.3419 sec.
イテレーション 7020 || Loss: 4.2364 || 10iter: 15.2389 sec.
イテレーション 7030 || Loss: 3.5943 || 10iter: 15.6650 sec.
イテレーション 7040 || Loss: 3.3698 || 10iter: 16.3619 sec.
イテレーション 7050 || Loss: 3.6750 || 10iter: 14.7767 sec.
イテレーション 7060 || Loss: 3.9623 || 10iter: 16.5807 sec.
イテレーション 7070 || Loss: 3.5282 || 10iter: 15.4715 sec.
イテレーション 7080 || Loss: 3.7662 || 10iter: 15.1593 sec.
イテレーション 7090 || Loss: 3.9443 || 10iter: 16.9625 sec.
イテレーション 7100 || Loss: 3.4090 || 10iter: 15.4385 sec.
イテレーション 7110 || Loss: 3.5168 || 10iter: 15.2522 sec.
イテレーション 7120 || Loss: 3.8373 || 10iter: 18.2417 sec.
イテレーション 7130 || Loss: 3.5320 || 10iter: 15.3971 sec.
イテレーション 7140 || Loss: 3.9199 || 10iter: 14.8943 sec.
イテレーション 7150 || Loss: 3.6482 || 10iter: 14.8117 sec.
イテレーション 7160 || Loss: 3.8150 || 10iter: 15.0710 sec.
-------------
（val）
-------------
epoch 40 || Epoch_TRAIN_Loss:651.2156 ||Epoch_VAL_Loss:713.8073
timer:  411.0267 sec.
-------------
Epoch 41/50
-------------
（train）
イテレーション 7170 || Loss: 3.3860 || 10iter: 18.1847 sec.
イテレーション 7180 || Loss: 4.0565 || 10iter: 15.4509 sec.
イテレーション 7190 || Loss: 3.1767 || 10iter: 16.1100 sec.
イテレーション 7200 || Loss: 3.2761 || 10iter: 17.1023 sec.
イテレーション 7210 || Loss: 3.8505 || 10iter: 15.3853 sec.
イテレーション 7220 || Loss: 3.7879 || 10iter: 17.0631 sec.
イテレーション 7230 || Loss: 3.4860 || 10iter: 15.6237 sec.
イテレーション 7240 || Loss: 3.1056 || 10iter: 15.7487 sec.
イテレーション 7250 || Loss: 3.4778 || 10iter: 16.4318 sec.
イテレーション 7260 || Loss: 3.1184 || 10iter: 15.0990 sec.
イテレーション 7270 || Loss: 3.4122 || 10iter: 17.4469 sec.
イテレーション 7280 || Loss: 3.5806 || 10iter: 15.4524 sec.
イテレーション 7290 || Loss: 2.8828 || 10iter: 15.3190 sec.
イテレーション 7300 || Loss: 3.6763 || 10iter: 16.4711 sec.
イテレーション 7310 || Loss: 3.5475 || 10iter: 15.5410 sec.
イテレーション 7320 || Loss: 3.9478 || 10iter: 16.7934 sec.
イテレーション 7330 || Loss: 3.7483 || 10iter: 14.6374 sec.
-------------
epoch 41 || Epoch_TRAIN_Loss:645.3319 ||Epoch_VAL_Loss:0.0000
timer:  298.7467 sec.
-------------
Epoch 42/50
-------------
（train）
イテレーション 7340 || Loss: 3.4850 || 10iter: 0.9958 sec.
イテレーション 7350 || Loss: 3.7107 || 10iter: 17.7339 sec.
イテレーション 7360 || Loss: 4.2470 || 10iter: 15.1688 sec.
イテレーション 7370 || Loss: 3.2640 || 10iter: 15.8900 sec.
イテレーション 7380 || Loss: 3.6967 || 10iter: 17.1975 sec.
イテレーション 7390 || Loss: 4.0349 || 10iter: 15.3755 sec.
イテレーション 7400 || Loss: 3.8139 || 10iter: 17.8316 sec.
イテレーション 7410 || Loss: 3.7668 || 10iter: 16.1779 sec.
イテレーション 7420 || Loss: 4.1276 || 10iter: 15.0719 sec.
イテレーション 7430 || Loss: 3.6707 || 10iter: 17.4658 sec.
イテレーション 7440 || Loss: 3.5222 || 10iter: 15.4710 sec.
イテレーション 7450 || Loss: 3.2614 || 10iter: 15.4360 sec.
イテレーション 7460 || Loss: 3.7876 || 10iter: 17.5334 sec.
イテレーション 7470 || Loss: 3.5073 || 10iter: 15.0761 sec.
イテレーション 7480 || Loss: 3.2084 || 10iter: 15.2841 sec.
イテレーション 7490 || Loss: 3.7473 || 10iter: 15.5549 sec.
イテレーション 7500 || Loss: 4.2402 || 10iter: 14.9226 sec.
イテレーション 7510 || Loss: 3.3902 || 10iter: 14.7707 sec.
-------------
epoch 42 || Epoch_TRAIN_Loss:644.5914 ||Epoch_VAL_Loss:0.0000
timer:  296.8113 sec.
-------------
Epoch 43/50
-------------
（train）
イテレーション 7520 || Loss: 3.6813 || 10iter: 2.5564 sec.
イテレーション 7530 || Loss: 3.7377 || 10iter: 15.4461 sec.
イテレーション 7540 || Loss: 3.8682 || 10iter: 14.7368 sec.
イテレーション 7550 || Loss: 3.3627 || 10iter: 14.6132 sec.
イテレーション 7560 || Loss: 3.6157 || 10iter: 14.8489 sec.
イテレーション 7570 || Loss: 3.5187 || 10iter: 15.4335 sec.
イテレーション 7580 || Loss: 3.8293 || 10iter: 15.0269 sec.
イテレーション 7590 || Loss: 3.8049 || 10iter: 15.1542 sec.
イテレーション 7600 || Loss: 3.4788 || 10iter: 14.8099 sec.
イテレーション 7610 || Loss: 3.9823 || 10iter: 15.5422 sec.
イテレーション 7620 || Loss: 3.9009 || 10iter: 15.1970 sec.
イテレーション 7630 || Loss: 3.2065 || 10iter: 15.3947 sec.
イテレーション 7640 || Loss: 3.4261 || 10iter: 15.2028 sec.
イテレーション 7650 || Loss: 3.4279 || 10iter: 14.9092 sec.
イテレーション 7660 || Loss: 3.6357 || 10iter: 14.9170 sec.
イテレーション 7670 || Loss: 3.9354 || 10iter: 15.0936 sec.
イテレーション 7680 || Loss: 3.4662 || 10iter: 15.0913 sec.
イテレーション 7690 || Loss: 3.2573 || 10iter: 14.9956 sec.
-------------
epoch 43 || Epoch_TRAIN_Loss:638.9081 ||Epoch_VAL_Loss:0.0000
timer:  281.3121 sec.
-------------
Epoch 44/50
-------------
（train）
イテレーション 7700 || Loss: 3.4286 || 10iter: 3.9793 sec.
イテレーション 7710 || Loss: 3.3710 || 10iter: 15.2360 sec.
イテレーション 7720 || Loss: 3.8949 || 10iter: 15.2642 sec.
イテレーション 7730 || Loss: 3.0223 || 10iter: 14.7707 sec.
イテレーション 7740 || Loss: 3.3859 || 10iter: 14.8882 sec.
イテレーション 7750 || Loss: 3.4736 || 10iter: 14.8446 sec.
イテレーション 7760 || Loss: 3.4282 || 10iter: 15.3853 sec.
イテレーション 7770 || Loss: 4.0607 || 10iter: 15.4582 sec.
イテレーション 7780 || Loss: 4.0463 || 10iter: 15.0903 sec.
イテレーション 7790 || Loss: 3.4988 || 10iter: 15.2194 sec.
イテレーション 7800 || Loss: 3.0332 || 10iter: 15.2160 sec.
イテレーション 7810 || Loss: 3.3415 || 10iter: 15.4401 sec.
イテレーション 7820 || Loss: 3.1721 || 10iter: 15.1435 sec.
イテレーション 7830 || Loss: 3.4395 || 10iter: 15.2865 sec.
イテレーション 7840 || Loss: 4.0262 || 10iter: 15.1774 sec.
イテレーション 7850 || Loss: 3.1724 || 10iter: 15.2310 sec.
イテレーション 7860 || Loss: 3.4387 || 10iter: 15.4886 sec.
イテレーション 7870 || Loss: 3.3752 || 10iter: 15.2976 sec.
-------------
epoch 44 || Epoch_TRAIN_Loss:638.0439 ||Epoch_VAL_Loss:0.0000
timer:  283.1804 sec.
-------------
Epoch 45/50
-------------
（train）
イテレーション 7880 || Loss: 3.4602 || 10iter: 5.6251 sec.
イテレーション 7890 || Loss: 3.6169 || 10iter: 14.8751 sec.
イテレーション 7900 || Loss: 3.3237 || 10iter: 14.9159 sec.
イテレーション 7910 || Loss: 3.3806 || 10iter: 15.2421 sec.
イテレーション 7920 || Loss: 3.6000 || 10iter: 14.9528 sec.
イテレーション 7930 || Loss: 3.1608 || 10iter: 15.1454 sec.
イテレーション 7940 || Loss: 3.9734 || 10iter: 15.3010 sec.
イテレーション 7950 || Loss: 3.6057 || 10iter: 15.3567 sec.
イテレーション 7960 || Loss: 3.3790 || 10iter: 15.1822 sec.
イテレーション 7970 || Loss: 3.9543 || 10iter: 15.2462 sec.
イテレーション 7980 || Loss: 3.8772 || 10iter: 15.1621 sec.
イテレーション 7990 || Loss: 3.8918 || 10iter: 15.3266 sec.
イテレーション 8000 || Loss: 3.7170 || 10iter: 15.1587 sec.
イテレーション 8010 || Loss: 3.4649 || 10iter: 14.8581 sec.
イテレーション 8020 || Loss: 3.1666 || 10iter: 14.7932 sec.
イテレーション 8030 || Loss: 3.9419 || 10iter: 15.2507 sec.
イテレーション 8040 || Loss: 3.8189 || 10iter: 15.3085 sec.
イテレーション 8050 || Loss: 3.5867 || 10iter: 15.5457 sec.
-------------
epoch 45 || Epoch_TRAIN_Loss:637.2015 ||Epoch_VAL_Loss:0.0000
timer:  282.6018 sec.
-------------
Epoch 46/50
-------------
（train）
イテレーション 8060 || Loss: 3.5221 || 10iter: 7.1946 sec.
イテレーション 8070 || Loss: 2.9708 || 10iter: 15.0524 sec.
イテレーション 8080 || Loss: 3.2402 || 10iter: 15.1549 sec.
イテレーション 8090 || Loss: 3.3481 || 10iter: 15.2737 sec.
イテレーション 8100 || Loss: 3.4358 || 10iter: 15.3100 sec.
イテレーション 8110 || Loss: 3.2831 || 10iter: 15.0489 sec.
イテレーション 8120 || Loss: 3.5305 || 10iter: 14.9348 sec.
イテレーション 8130 || Loss: 3.1732 || 10iter: 15.1294 sec.
イテレーション 8140 || Loss: 3.2429 || 10iter: 15.3319 sec.
イテレーション 8150 || Loss: 3.6802 || 10iter: 14.8812 sec.
イテレーション 8160 || Loss: 3.3289 || 10iter: 15.3531 sec.
イテレーション 8170 || Loss: 3.6176 || 10iter: 15.2629 sec.
イテレーション 8180 || Loss: 3.7495 || 10iter: 15.2598 sec.
イテレーション 8190 || Loss: 3.2969 || 10iter: 14.9126 sec.
イテレーション 8200 || Loss: 3.5836 || 10iter: 14.8904 sec.
イテレーション 8210 || Loss: 3.2398 || 10iter: 14.9859 sec.
イテレーション 8220 || Loss: 4.0211 || 10iter: 15.3772 sec.
イテレーション 8230 || Loss: 3.5552 || 10iter: 14.9590 sec.
-------------
epoch 46 || Epoch_TRAIN_Loss:633.0747 ||Epoch_VAL_Loss:0.0000
timer:  281.9943 sec.
-------------
Epoch 47/50
-------------
（train）
イテレーション 8240 || Loss: 3.5094 || 10iter: 8.8625 sec.
イテレーション 8250 || Loss: 3.7752 || 10iter: 15.2578 sec.
イテレーション 8260 || Loss: 3.6005 || 10iter: 14.7238 sec.
イテレーション 8270 || Loss: 3.7463 || 10iter: 15.2771 sec.
イテレーション 8280 || Loss: 3.3010 || 10iter: 15.1956 sec.
イテレーション 8290 || Loss: 3.7087 || 10iter: 15.1413 sec.
イテレーション 8300 || Loss: 3.7898 || 10iter: 14.8021 sec.
イテレーション 8310 || Loss: 3.5674 || 10iter: 15.1326 sec.
イテレーション 8320 || Loss: 3.2634 || 10iter: 15.2320 sec.
イテレーション 8330 || Loss: 2.9950 || 10iter: 15.1166 sec.
イテレーション 8340 || Loss: 3.7282 || 10iter: 14.9029 sec.
イテレーション 8350 || Loss: 3.6867 || 10iter: 14.9961 sec.
イテレーション 8360 || Loss: 3.4366 || 10iter: 15.1436 sec.
イテレーション 8370 || Loss: 3.5840 || 10iter: 15.2412 sec.
イテレーション 8380 || Loss: 3.1400 || 10iter: 15.1166 sec.
イテレーション 8390 || Loss: 3.1535 || 10iter: 15.1204 sec.
イテレーション 8400 || Loss: 3.4755 || 10iter: 15.1245 sec.
イテレーション 8410 || Loss: 3.6955 || 10iter: 15.3623 sec.
-------------
epoch 47 || Epoch_TRAIN_Loss:622.3093 ||Epoch_VAL_Loss:0.0000
timer:  281.8193 sec.
-------------
Epoch 48/50
-------------
（train）
イテレーション 8420 || Loss: 3.7192 || 10iter: 10.3414 sec.
イテレーション 8430 || Loss: 3.4561 || 10iter: 14.8063 sec.
イテレーション 8440 || Loss: 4.4070 || 10iter: 15.1951 sec.
イテレーション 8450 || Loss: 3.4718 || 10iter: 15.3511 sec.
イテレーション 8460 || Loss: 3.3853 || 10iter: 15.5343 sec.
イテレーション 8470 || Loss: 3.3205 || 10iter: 15.4708 sec.
イテレーション 8480 || Loss: 2.6448 || 10iter: 14.8815 sec.
イテレーション 8490 || Loss: 3.5758 || 10iter: 14.8033 sec.
イテレーション 8500 || Loss: 4.0023 || 10iter: 14.7393 sec.
イテレーション 8510 || Loss: 3.9702 || 10iter: 15.4206 sec.
イテレーション 8520 || Loss: 3.7118 || 10iter: 15.1948 sec.
イテレーション 8530 || Loss: 3.6073 || 10iter: 14.9562 sec.
イテレーション 8540 || Loss: 3.4305 || 10iter: 14.7808 sec.
イテレーション 8550 || Loss: 3.4446 || 10iter: 15.7059 sec.
イテレーション 8560 || Loss: 3.6277 || 10iter: 15.3274 sec.
イテレーション 8570 || Loss: 3.8712 || 10iter: 14.8921 sec.
イテレーション 8580 || Loss: 3.7294 || 10iter: 15.1275 sec.
イテレーション 8590 || Loss: 3.2317 || 10iter: 15.3630 sec.
-------------
epoch 48 || Epoch_TRAIN_Loss:625.0028 ||Epoch_VAL_Loss:0.0000
timer:  282.4109 sec.
-------------
Epoch 49/50
-------------
（train）
イテレーション 8600 || Loss: 3.3111 || 10iter: 12.2228 sec.
イテレーション 8610 || Loss: 3.8078 || 10iter: 14.7367 sec.
イテレーション 8620 || Loss: 3.4539 || 10iter: 14.6185 sec.
イテレーション 8630 || Loss: 3.7029 || 10iter: 14.8554 sec.
イテレーション 8640 || Loss: 3.1350 || 10iter: 15.2491 sec.
イテレーション 8650 || Loss: 3.6857 || 10iter: 14.9005 sec.
イテレーション 8660 || Loss: 3.4549 || 10iter: 15.2806 sec.
イテレーション 8670 || Loss: 3.4849 || 10iter: 15.0353 sec.
イテレーション 8680 || Loss: 3.2488 || 10iter: 14.8302 sec.
イテレーション 8690 || Loss: 3.8561 || 10iter: 14.8566 sec.
イテレーション 8700 || Loss: 3.4506 || 10iter: 15.3013 sec.
イテレーション 8710 || Loss: 3.4223 || 10iter: 15.4015 sec.
イテレーション 8720 || Loss: 2.9792 || 10iter: 14.8199 sec.
イテレーション 8730 || Loss: 3.6155 || 10iter: 14.6798 sec.
イテレーション 8740 || Loss: 3.0266 || 10iter: 15.2551 sec.
イテレーション 8750 || Loss: 3.9916 || 10iter: 15.3232 sec.
イテレーション 8760 || Loss: 3.4299 || 10iter: 14.6586 sec.
イテレーション 8770 || Loss: 3.6696 || 10iter: 14.4989 sec.
-------------
epoch 49 || Epoch_TRAIN_Loss:615.1798 ||Epoch_VAL_Loss:0.0000
timer:  279.3445 sec.
-------------
Epoch 50/50
-------------
（train）
イテレーション 8780 || Loss: 4.4273 || 10iter: 13.7506 sec.
イテレーション 8790 || Loss: 3.6891 || 10iter: 15.0468 sec.
イテレーション 8800 || Loss: 3.5288 || 10iter: 15.1564 sec.
イテレーション 8810 || Loss: 3.4939 || 10iter: 14.9161 sec.
イテレーション 8820 || Loss: 3.5610 || 10iter: 15.1930 sec.
イテレーション 8830 || Loss: 3.4852 || 10iter: 15.3911 sec.
イテレーション 8840 || Loss: 3.8326 || 10iter: 14.9988 sec.
イテレーション 8850 || Loss: 3.3196 || 10iter: 14.8499 sec.
イテレーション 8860 || Loss: 3.3834 || 10iter: 14.9815 sec.
イテレーション 8870 || Loss: 3.5135 || 10iter: 15.4150 sec.
イテレーション 8880 || Loss: 2.5664 || 10iter: 15.3391 sec.
イテレーション 8890 || Loss: 3.4200 || 10iter: 15.0390 sec.
イテレーション 8900 || Loss: 3.6053 || 10iter: 15.2844 sec.
イテレーション 8910 || Loss: 3.0087 || 10iter: 14.7189 sec.
イテレーション 8920 || Loss: 3.1951 || 10iter: 14.8307 sec.
イテレーション 8930 || Loss: 3.5759 || 10iter: 14.9867 sec.
イテレーション 8940 || Loss: 3.5825 || 10iter: 14.9940 sec.
イテレーション 8950 || Loss: 3.6784 || 10iter: 14.7527 sec.
-------------
（val）
-------------
epoch 50 || Epoch_TRAIN_Loss:609.9108 ||Epoch_VAL_Loss:684.5518
timer:  388.3606 sec.